Koden under er en versjon som kjører synkront.

In [ ]:
import time

def make_meal(delay, what):
    print(what, end=": ")
    time.sleep(delay)
    print('Ferdig!')

def main():
    print(f"Startet {time.strftime('%X')}")
    
    make_meal(3, 'Lage kaffe')
    make_meal(5, 'Koke egg')
    make_meal(3, 'Steike bacon')
    make_meal(5, 'Varme bønner i tomatsaus')
    make_meal(3, 'Riste brød')
    make_meal(1, 'Sette på bordet')

    print('Frokosten er ferdig!')
    print(f"Ferdig ved {time.strftime('%X')}")

main()

Og nå benytter vi oss av asyncio for å asynkront kjøre jobbene i parallell

In [ ]:
import asyncio
import time

async def make_meal(delay, what):
    await asyncio.sleep(delay)
    print(what, ": Ferdig!")

async def main() :

    print(f"Startet {time.strftime('%X')}")
    await asyncio.gather(make_meal(3, 'Lage kaffe'),
        make_meal(5, 'Koke egg'),
        make_meal(3, 'Steike bacon'),
        make_meal(5, 'Varme bønner i tomatsaus'),
        make_meal(3, 'Riste brød'),
        make_meal(1, 'Sette på bordet'))
    print(f"Ferdig ved {time.strftime('%X')}")

await main()


Skal man utveksle informasjon på tvers av jobber er det naturlig å gjøre dette via en kø. Det finnes tråd-sikre måter å gjøre dette på.

In [ ]:
import asyncio
import random
import time


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await asyncio.sleep(sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


await main()

Eksempel på producer/consumer applikasjon: Kjøkken på restaurant. Restauranten stenger når kokkene har greid å tømme ordre-boka.

In [57]:
import asyncio
import random
import time

class Order :
    def __init__(self, meal, howLong) :
        self.__meal = meal
        self.__howLong = howLong
    
    @property
    def meal(self) :
        return self.__meal
    
    @property
    def howLong(self) :
        return self.__howLong
      

async def waiter(meal, how_long, queue) :
    # Place order in the queue
    while True :
        print("Legger matrett av typen", meal, "inn til bestilling")
        await queue.put(Order(meal, how_long))
        await asyncio.sleep(random.uniform(0,10))

async def cheaf(queue) :
    while not queue.empty() :
        order = await queue.get()
        print("Lager matrett av type", order.meal, "ved å bruke", order.howLong, "sekunder")
        await asyncio.sleep(order.howLong)
        print("Ferdig med å lage", order.meal)
        queue.task_done()

    print("**** Vi stenger kjøkkenet! *****")



async def main() :
    queue = asyncio.Queue(10)

    kelner1 = asyncio.create_task(waiter("Kjøttsuppe",1, queue))
    kelner2 = asyncio.create_task(waiter("Biff",2, queue))
    kelner3 = asyncio.create_task(waiter("Grøt",3, queue))
  
    consumer = asyncio.create_task(cheaf(queue))
    tasks = [kelner1, kelner2, kelner3, consumer]

    await queue.join()
    await asyncio.gather(*tasks)
#    kelner1.cancel()
#    kelner2.cancel()
#    kelner3.cancel()
    print('====')

await main()



    



    
    


Task was destroyed but it is pending!
task: <Task pending name='Task-180' coro=<waiter() done, defined at C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/1970018282.py:19> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task pending name='Task-181' coro=<waiter() done, defined at C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/1970018282.py:19> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task pending name='Task-182' coro=<waiter() done, defined at C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/1970018282.py:19> wait_for=<Future cancelled>>


Legger matrett av typen Kjøttsuppe inn til bestilling
Legger matrett av typen Biff inn til bestilling
Legger matrett av typen Grøt inn til bestilling
Lager matrett av type Kjøttsuppe ved å bruke 1 sekunder
Legger matrett av typen Kjøttsuppe inn til bestilling
Ferdig med å lage Kjøttsuppe
Lager matrett av type Biff ved å bruke 2 sekunder
Ferdig med å lage Biff
Lager matrett av type Kjøttsuppe ved å bruke 2 sekunder
Ferdig med å lage Biff
Lager matrett av type Grøt ved å bruke 3 sekunder
Legger matrett av typen Grøt inn til bestilling
Ferdig med å lage Kjøttsuppe
Lager matrett av type Grøt ved å bruke 4 sekunder
Legger matrett av typen Biff inn til bestilling
Ferdig med å lage Grøt
Lager matrett av type Biff ved å bruke 2 sekunder
Legger matrett av typen Grøt inn til bestilling
Legger matrett av typen Biff inn til bestilling
Ferdig med å lage Biff
Lager matrett av type Grøt ved å bruke 3 sekunder
Legger matrett av typen Kjøttsuppe inn til bestilling
Ferdig med å lage Grøt
Lager matrett a

Task exception was never retrieved
future: <Task finished name='Task-143' coro=<waiter() done, defined at C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/2234202621.py:19> exception=TypeError('Random.random() takes no arguments (1 given)')>
Traceback (most recent call last):
  File "C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/2234202621.py", line 24, in waiter
    await asyncio.sleep(random.random(10))
TypeError: Random.random() takes no arguments (1 given)
Task exception was never retrieved
future: <Task finished name='Task-144' coro=<waiter() done, defined at C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/2234202621.py:19> exception=TypeError('Random.random() takes no arguments (1 given)')>
Traceback (most recent call last):
  File "C:\Users\hfr019\AppData\Local\Temp/ipykernel_22124/2234202621.py", line 24, in waiter
    await asyncio.sleep(random.random(10))
TypeError: Random.random() takes no arguments (1 given)
Task exception was never retrieved
future: <Task finishe

CancelledError: 

Legger matrett av typen Grøt inn til bestilling
Ferdig med å lage Biff
Lager matrett av type Grøt ved å bruke 4 sekunder
Legger matrett av typen Biff inn til bestilling
Ferdig med å lage Grøt
Lager matrett av type Kjøttsuppe ved å bruke 2 sekunder
Ferdig med å lage Kjøttsuppe
Lager matrett av type Grøt ved å bruke 4 sekunder
Ferdig med å lage Grøt
Lager matrett av type Biff ved å bruke 3 sekunder
Legger matrett av typen Biff inn til bestilling
Ferdig med å lage Biff
Lager matrett av type Grøt ved å bruke 4 sekunder
Legger matrett av typen Kjøttsuppe inn til bestilling
Legger matrett av typen Grøt inn til bestilling
